In [1]:
!nvidia-smi

Fri Jul 12 15:41:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:41:00.0 Off |                    0 |
|  0%   28C    P8             24W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
original_data_path = '../data/force_data/data.json' # path to raw data
calculator_path = '../configs/calculator/config_morse.yml' # path to calculator config
rank = 0 # device Nr.
batch_size = 16
save_dir = '../data/silica_optimization' # path to save augmented data
perturb_strength = [0.5] # perturbation strength
task = "optim" # optimization doesn't need labels
n_round_augmentation = len(perturb_strength)
calculated_properties = ('energy', 'forces', 'stress')

In [29]:
import json

with open(original_data_path, 'r') as f:
    data = json.load(f)
    
len(data)

2864

In [30]:
import numpy as np

structures = [{key: np.array(val) if isinstance(val, list) else val for key, val in dt.items()} for dt in data]

In [31]:
orig_forces = np.concatenate([structure['forces'] for structure in structures])
force_magnitude = np.linalg.norm(orig_forces, axis=1)
force_magnitude.shape

(266858,)

In [32]:
import matplotlib.pyplot as plt

plt.hist(force_magnitude, bins=1000)
plt.title(f"Mean: {np.mean(force_magnitude):.2f}, Std: {np.std(force_magnitude):.2f}, Max: {np.max(force_magnitude):.2f}")
plt.xlim(left=0, right=500)
plt.ylim(bottom=0, top=1.2*10e6)
plt.show();

In [33]:
import numpy as np

def perturb_positions(pos, distance):
    random_vectors = np.random.randn(pos.shape[0], 3)
    random_vectors /= np.linalg.norm(random_vectors, axis=1, keepdims=True)
    perturbation_distances = np.full((pos.shape[0],), distance)
    return pos + perturbation_distances.reshape(-1, 1) * random_vectors

In [34]:
from itertools import chain

augmented_structure_list = [list() for _ in range(n_round_augmentation)]

for i in range(n_round_augmentation):
    for s in structures:
        aug = s.copy()
        aug['positions'] = perturb_positions(s['positions'], perturb_strength[i])
        augmented_structure_list[i].append(aug)
        
augmented_structures = list(chain.from_iterable(augmented_structure_list))
len(augmented_structures)

2864

In [35]:
# For optimization
structure_to_write = []
if task == 'optim':
    for s in augmented_structures:
        structure_to_write.append({
            'structure_id': s['structure_id'],
            'positions': s['positions'].tolist(),
            'cell': s['cell'].tolist(),
            'atomic_numbers': s['atomic_numbers'].tolist()
        })
    json.dump(structure_to_write, open(f'{save_dir}/data.json', 'w'))
    

In [44]:
from typing import List
from ase import Atoms

if task != "optim":
    atoms_list: List[Atoms] = [
        Atoms(
            symbols=s['atomic_numbers'],
            positions=s['positions'],
            cell=s['cell']
        )
        for s in augmented_structures
    ]

In [45]:
from matdeeplearn.common.ase_utils import MDLCalculator

if task != "optim":
    calc = MDLCalculator(calculator_path, rank=rank)

INFO:root:MDLCalculator instantiated from config: ../configs/calculator/config_morse.yml


INFO:root:MDLCalculator: setting up Morse for calculation
INFO:root:MDLCalculator: weights for model No.1 loaded from ../results/2024-06-08-00-44-29-529-morse_mp/checkpoint_0/best_checkpoint.pt


In [46]:
import torch
from torch_geometric.loader import DataLoader

if task != "optim":
    data_list = [
        calc.atoms_to_data_object(atoms) for atoms in atoms_list
    ]

    results = {prop: [] for prop in calculated_properties}
    results.update(
        {
            'structure_id': [s['structure_id'] for s in augmented_structures],
            'positions': [s['positions'] for s in augmented_structures],
            'cell': [s['cell'] for s in augmented_structures],
            'atomic_numbers': [s['atomic_numbers'] for s in augmented_structures],
        }
    )

In [47]:
from tqdm import tqdm

if task != "optim":
    loader = DataLoader(data_list, batch_size=batch_size)
    loader_iter = iter(loader)

    for idx, batch in tqdm(enumerate(loader_iter), total=len(loader_iter)):
        batch = batch.to(f'cuda:{rank}')
        
        out_list = []
        for model in calc.models:   
            model.eval()
            out_list.append(model(batch))

        energy = torch.stack([entry["output"] for entry in out_list]).mean(dim=0).detach().cpu().numpy().flatten().tolist()
        forces = torch.stack([entry["pos_grad"] for entry in out_list]).mean(dim=0)
        stresses = torch.stack([entry["cell_grad"] for entry in out_list]).mean(dim=0)
        
        forces_each_structure = []
        stress_each_structure = []
        
        for i in range(len(batch)):
            split_force = forces[torch.where(batch.batch == i)]
            forces_each_structure.append(split_force.detach().cpu().numpy().tolist())
            stress_each_structure.append(stresses[i].detach().cpu().numpy().tolist())
        
        results['energy'].extend(energy)
        results['forces'].extend(forces_each_structure)
        results['stress'].extend(stress_each_structure)

100%|██████████| 58653/58653 [09:31<00:00, 102.65it/s] 


In [48]:
import matplotlib.pyplot as plt

if task != "optim":
    for key in results.keys():
        print(key, ':', len(results[key]))
        
    forces_arr = np.concatenate(results['forces'], axis=0)
    force_magnitude = np.linalg.norm(forces_arr, axis=1)
    
    plt.hist(force_magnitude, bins=1000)
    plt.title(f"Mean: {np.mean(force_magnitude):.2f}, Std: {np.std(force_magnitude):.2f}, Max: {np.max(force_magnitude):.2f}")
    plt.xlim(left=0, right=500)
    plt.ylim(bottom=0, top=1.2*10e6)
    plt.show();

energy : 938435
forces : 938435
stress : 938435
structure_id : 938435
positions : 938435
cell : 938435
atomic_numbers : 938435


In [51]:
import os

if task != "optim":
    os.makedirs(save_dir, exist_ok=True)
    saved_data = [
        {
            'structure_id': results['structure_id'][i],
            'positions': results['positions'][i].tolist(),
            'cell': results['cell'][i].tolist(),
            'atomic_numbers': results['atomic_numbers'][i].tolist(),
            'y': results['energy'][i],
            'energy': results['energy'][i],
            'stress': [results['stress'][i]],
            'forces': results['forces'][i],
        } for i in range(len(results['structure_id']))
    ]

    with open(f'{save_dir}/data.json', 'w') as f:
        json.dump(saved_data, f)
        
    print(f'{len(saved_data)} structures saved')